In [1]:
from dataloader import *
from  config import *
from data_processor import preprocess_data

Using TensorFlow backend.


In [2]:
data = DataLoader(config_mixed_coco2014_coco2014)

In [3]:
data= preprocess_data(data)

Descriptions cleaned.
[['a', 'woman', 'wearing', 'a', 'net', 'on', 'her', 'head', 'cutting', 'a', 'cake'], ['a', 'woman', 'cutting', 'a', 'large', 'white', 'sheet', 'cake'], ['a', 'woman', 'wearing', 'a', 'hair', 'net', 'cutting', 'a', 'large', 'sheet', 'cake'], ['there', 'is', 'a', 'woman', 'that', 'is', 'cutting', 'a', 'white', 'cake'], ['a', 'woman', 'marking', 'a', 'cake', 'with', 'the', 'back', 'of', 'a', 'chefs', 'knife']]
Descriptions wraped into start and stop words.
['START a woman wearing a net on her head cutting a cake STOP', 'START a woman cutting a large white sheet cake STOP', 'START a woman wearing a hair net cutting a large sheet cake STOP', 'START there is a woman that is cutting a white cake STOP', 'START a woman marking a cake with the back of a chefs knife STOP']


2022-09-07 18:33:43.736994: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-07 18:33:43.737466: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 16. Tune using inter_op_parallelism_threads for best performance.


Processed:
100
Processed:
200
Processed:
300
Processed:
400
Processed:
500
Processed:
600
Processed:
700
Processed:
800
Processed:
900
Processed:
1000
Processed:
1100
Processed:
1200
Processed:
1300
Processed:
1400
Processed:
1500
Processed:
1600
Processed:
1700
Processed:
1800
Processed:
1900
Processed:
2000
Processed:
2100
Processed:
2200
Processed:
2300
Processed:
2400
Processed:
2500
Processed:
2600
Processed:
2700
Processed:
2800
Processed:
2900
Processed:
3000
Processed:
3100
Processed:
3200
Processed:
3300
Processed:
3400
Processed:
3500
Processed:
3600
Processed:
3700
Processed:
3800
Processed:
3900
Processed:
4000
Processed:
4100
Processed:
4200
Processed:
4300
Processed:
4400
Processed:
4500
Processed:
4600
Processed:
4700
Processed:
4800
Processed:
4900
Processed:
5000
Processed:
5100
Processed:
5200
Processed:
5300
Processed:
5400
Processed:
5500
Processed:
5600
Processed:
5700
Processed:
5800
Processed:
5900
Processed:
6000
Processed:
6100
Processed:
6200
Processed:
6300
P

Processed:
38700
Processed:
38800
Processed:
38900
Processed:
39000
Processed:
39100
Processed:
39200
Processed:
39300
Processed:
39400
Processed:
39500
Processed:
39600
Processed:
39700
Processed:
39800
Processed:
39900
Processed:
40000
Processed:
40100
Processed:
40200
Processed:
40300
Processed:
40400
Processed:
40500
Processed:
40600
Processed:
40700
Processed:
40800
Processed:
40900
Processed:
41000
Processed:
41100
Processed:
41200
Processed:
41300
Processed:
41400
Processed:
41500
Processed:
41600
Processed:
41700
Processed:
41800
Processed:
41900
Processed:
42000
Processed:
42100
Processed:
42200
Processed:
42300
Processed:
42400
Processed:
42500
Processed:
42600
Processed:
42700
Processed:
42800
Processed:
42900
Processed:
43000
Processed:
43100
Processed:
43200
Processed:
43300
Processed:
43400
Processed:
43500
Processed:
43600
Processed:
43700
Processed:
43800
Processed:
43900
Processed:
44000
Processed:
44100
Processed:
44200
Processed:
44300
Processed:
44400
Processed:
445

Processed:
86900
Processed:
87000
Processed:
87100
Processed:
87200
Processed:
87300
Processed:
87400
Processed:
87500
Processed:
87600
Processed:
87700
Processed:
87800
Processed:
87900
Processed:
88000
Processed:
88100
Processed:
88200
Processed:
88300
Processed:
88400
Processed:
88500
Processed:
88600
Processed:
88700
Processed:
88800
Processed:
88900
Processed:
89000
Processed:
89100
Processed:
89200
Processed:
89300
Processed:
89400
Processed:
89500
Processed:
89600
Processed:
89700
Processed:
89800
Processed:
89900
Processed:
90000
Processed:
90100
Processed:
90200
Processed:
90300
Processed:
90400
Processed:
90500
Processed:
90600
Processed:
90700
Processed:
90800
Processed:
90900
Processed:
91000
Processed:
91100
Processed:
91200
Processed:
91300
Processed:
91400
Processed:
91500
Processed:
91600
Processed:
91700
Processed:
91800
Processed:
91900
Processed:
92000
Processed:
92100
Processed:
92200
Processed:
92300
Processed:
92400
Processed:
92500
Processed:
92600
Processed:
927

In [4]:
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, \
    Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers.merge import add
from keras.models import Model
from keras import Input, layers
from keras import callbacks
from eval_utils import calculate_results, prepare_for_evaluation
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from numpy import array
import pickle
from eval_utils import generate_report

In [5]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch, vocab_size):
    X1, X2, y = list(), list(), list()
    n = 0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n += 1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n == num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n = 0

class ModelImpl:
    def __init__(self, data):
        self.data=data
        inputs1 = Input(shape=(2048,))
        fe1 = Dropout(0.5)(inputs1)
        fe2 = Dense(256, activation='relu')(fe1)
        inputs2 = Input(shape=(self.data.max_length,))
        se1 = Embedding(self.data.vocab_size, general[self.data.language]["embedings_dim"], mask_zero=True)(inputs2)
        se2 = Dropout(0.5)(se1)
        se3 = LSTM(256)(se2)
        decoder1 = add([fe2, se3])
        decoder2 = Dense(256, activation='relu')(decoder1)
        outputs = Dense(self.data.vocab_size, activation='softmax')(decoder2)
        self.model = Model(inputs=[inputs1, inputs2], outputs=outputs)
        self.model.summary()
        self.model.layers[2]

        self.model.layers[2].set_weights([self.data.embedding_matrix])
        self.model.layers[2].trainable = False

        self.model.compile(loss='categorical_crossentropy', optimizer=self.optimizer())
        self.setup()

    def optimizer(self):
        return Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    def setup(self):
        # model.optimizer.lr = 0.001
        self.epochs = 2
        self.number_pics_per_bath = 100
        self.steps = len(self.data.train_captions_wrapped) // self.number_pics_per_bath

    def train(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        if self.data.configuration["train_model"]:
            callback = callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
            generator = data_generator(self.data.train_captions_wrapped,
                                       self.data.encoded_images_train,
                                       self.data.wordtoix,
                                       self.data.max_length,
                                       self.number_pics_per_bath,
                                       self.data.vocab_size)
            self.model.fit(generator, epochs=self.epochs,
                           steps_per_epoch=self.steps,
                           callbacks=[callback],
                           verbose=1)
            if self.data.configuration["save_model"]:
                writepath = model_weights_path+ "/"+'model' + '.h5'
                self.model.save(writepath)
                self.model.save_weights(model_weights_path
                                        + self.data.configuration["model_save_path"])
        else:
            self.model.load_weights(model_weights_path
                                        +self.data.configuration["model_save_path"])

    def evaluate(self):
        expected, results = prepare_for_evaluation(self.data.encoded_images_test, self.data, self.model)
        with open("expected.pkl","wb") as encoded_pickle:
            pickle.dump(expected, encoded_pickle)
        with open("results.pkl","wb") as encoded_pickle2:
            pickle.dump(results, encoded_pickle2)
#         with open("expected.pkl", 'rb') as encoded_pickle:
#             expected = pickle.load(encoded_pickle)
#         with open("results.pkl", 'rb') as encoded_pickle:
#             results = pickle.load(encoded_pickle)
#         out = calculate_results(expected, results, self.data.configuration)
        print(out)

model=ModelImpl(data)


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 51)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 51, 199)      1451307     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.train()

Epoch 1/2
 389/1132 [=========>....................] - ETA: 3:06:50 - loss: 4.2841

In [ ]:
model.evaluate()

In [2]:
generate_report(general["results_directory"])


NameError: name 'generate_report' is not defined